In [1]:
!pip install -qq openai==0.28

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 2.7 MB/s eta 0:00:00


In [2]:
!pip install -qq transformers

In [3]:
!pip install -qq pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 27.0 MB/s eta 0:00:00


In [4]:
!pip install python-docx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 3.6 MB/s eta 0:00:00


In [5]:
!pip install -qq transformers datasets accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.0 MB/s eta 0:00:00


In [6]:
#install quietly
!pip install huggingface_hub -qq

In [7]:
import openai
from transformers import T5Tokenizer, T5ForConditionalGeneration
from transformers import pipeline
from huggingface_hub import login
from typing import List
import torch
import gc

import os

try:
    from google.colab import drive, userdata
    COLAB = True
    print("Note: using Google CoLab")
except:
    print("Note: not using Google CoLab")
    COLAB = False

openai.api_key = userdata.get('OPENAI_API_KEY')

Note: using Google CoLab


In [8]:
# Ensure that you export the environment variable before running the code
# HF_KEY_FINE = userdata.get("HF_KEY_FINE")  # Set this with your actual token before running
# login(token=os.environ.get("HF_KEY_FINE"))

In [9]:
from google.colab import userdata
HF_TOKEN = userdata.get('HF_ACCESS_TOKEN')

In [10]:
# Function to generate text using OpenAI API
def openai_generate_text(prompt, api_key, model="gpt-4", max_tokens=100, temperature=0.7):
    openai.api_key = api_key

    try:
        # Call OpenAI's GPT model to generate text using Chat API
        response = openai.ChatCompletion.create(
            model=model,
            messages=[{"role": "user", "content": prompt}],
            max_tokens=max_tokens,
            temperature=temperature
        )

        # Extract generated text from response
        generated_text = response['choices'][0]['message']['content'].strip()
        return generated_text

    except Exception as e:
        print(f"An error occurred: {e}")
        return None

In [11]:
from transformers import AutoTokenizer, AutoModelForCausalLM

In [12]:
def gemma_generate_text(prompt, model="google/gemma-2b-it", max_tokens=100,):

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print("Device:", device)

    # Use pipeline API
    pipe = pipeline(
        "text-generation",
        model=model,
        model_kwargs={"torch_dtype": "auto"},
        device=device,
    )

    # Delete large objects
    del model
    gc.collect()
    torch.cuda.empty_cache()

    # Generate the text with a maximum of 256 new tokens
    response = pipe(prompt, max_new_tokens=max_tokens)

    # Extract the generated text from the response
    generated_response = response[0]["generated_text"].strip()
    return generated_response

In [13]:
# Load the T5 model and tokenizer
def load_model_and_tokenizer(model_path):
    tokenizer = T5Tokenizer.from_pretrained("google/t5-v1_1-base")
    model = T5ForConditionalGeneration.from_pretrained(model_path)
    return tokenizer, model

In [14]:
# Load the Gemma model and tokenizer
def load_gemma_model_and_tokenizer(model_path):
    login(token=HF_TOKEN)
    tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b-it")
    model = AutoModelForCausalLM.from_pretrained(model_path)
    return tokenizer, model

In [26]:
# Generate text using the model based on the given prompt
def generate_text(prompt, tokenizer, model):
    input_ids = tokenizer(prompt, return_tensors="pt", padding=True).input_ids
    outputs = model.generate(input_ids, max_length=512, num_return_sequences=1)
    # outputs = model.generate(input_ids, max_length=512, num_return_sequences=1, encoder_outputs=None)

    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_text


In [16]:
# Function to extract relevant key words/phrases from response
def extract_key_words(response: str) -> List[str]:
    key_words = [word.strip() for word in response.split() if len(word) > 3]
    return key_words

# Function to extract key words/phrases from job description
def extract_key_phrases(job_description: str) -> List[str]:
    key_words = [word.strip() for word in job_description.split() if len(word) > 3]
    return key_words

In [17]:
# Function to find matching experience in resume text
# def match_experience_in_resume(key_words: List[str], resume_txt: str) -> List[str]:
#     matched_experiences = set()
#     for word in key_words:
#         if word.lower() in resume_txt.lower():
#             # Find sentence containing the word in resume
#             sentences = resume_txt.split(". ")
#             for sentence in sentences:
#                 if word.lower() in sentence.lower():
#                     matched_experiences.add(sentence.strip())
#     return list(matched_experiences)


# Function to find matching experience in resume text based on job description key phrases
def match_experience_to_job(key_phrases: List[str], resume_txt: str) -> List[str]:
    matched_experiences = set()
    sentences = resume_txt.split(". ")

    for sentence in sentences:
        for phrase in key_phrases:
            if phrase.lower() in sentence.lower():
                matched_experiences.add(sentence.strip())
                break
    return list(matched_experiences)

In [18]:
# Format the matched experiences into bullet points
def format_bullet_points(experiences: List[str]) -> str:
    if not experiences:
        return "No matching experience found."
    bullet_points = "\n".join([f"- {exp}" for exp in experiences])
    return bullet_points

In [19]:
#Summarize Resume
import pdfplumber
from transformers import pipeline

# Function to extract text from PDF
def extract_text_from_pdf(pdf_path: str) -> str:
    all_text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            all_text += page.extract_text()
    return all_text.strip()

# Function to summarize the extracted text using HuggingFace T5 model
def summarize_text(text: str, max_length: int = 150) -> str:
    summarizer = pipeline("summarization", model="t5-base", tokenizer="t5-base")
    summary = summarizer(text, max_length=max_length, min_length=30, do_sample=False)
    return summary[0]['summary_text']

# Using OpenAI API for summarization
def summarize_experience(text: str, api_key: str, model="gpt-4"):
    openai.api_key = api_key
    try:
        response = openai.ChatCompletion.create(
            model=model,
            # messages=[{"role": "user", "content": f"Summarize the work experience in this resume:\n{text}"}],
            messages=[{"role": "user", "content": f"Summarize the work experience for each job position held in this resume:\n{text}"}],
            max_tokens=300
        )
        summary = response['choices'][0]['message']['content'].strip()
        return summary
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

# Extract and summarize resume from PDF
def process_resume(pdf_path: str, selection: str, api_key: str):
    summarized_text = None
    resume_text = extract_text_from_pdf(pdf_path)
    #print("Extracted Resume Text:\n", resume_text)

    if selection == "openai":
        summarized_text = summarize_experience(resume_text, api_key)
    elif selection == "huggingface":
        summarized_text = summarize_text(resume_text)
    else:
        print("Invalid selection. Please choose 'openai' or 'huggingface'.")

    print("\nSummarized Resume Text:\n", summarized_text)
    return summarized_text


pdf_path = "Pauline Korukundo CV.pdf"

In [20]:
from docx import Document
from docx.shared import Pt, Inches
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT
from docx.shared import Cm
import re
import spacy

In [21]:
# Use SpaCy to Extract Job Titles

# Load spaCy model
nlp = spacy.load("en_core_web_sm")

# Function to check if a line is a job title using NER
def is_job_title(line: str) -> bool:
    doc = nlp(line)
    for ent in doc.ents:
        if ent.label_ == "ORG" or ent.label_ == "JOB_TITLE":
            return True
    return False

Prepare output document

In [30]:
def create_resume_doc(bullet_points: str, education: str, profile: str, file_name: str = "Tailored_Resume.docx"):
    # Create a new Document
    doc = Document()

    # Set the document to A4 size
    section = doc.sections[0]
    section.page_height = Cm(29.7)
    section.page_width = Cm(21.0)

    # Set margins for A4
    section.left_margin = Cm(2)
    section.right_margin = Cm(2)
    section.top_margin = Cm(2)
    section.bottom_margin = Cm(2)

    # Title of the Resume
    title = doc.add_heading("RESUME", level=1)
    title.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER

    # Profile Section

    if profile:
        doc.add_paragraph("Profile", style='Heading 2')
        achievement_lines = profile.split("\n")
        for line in achievement_lines:
            if line.strip():
                doc.add_paragraph(line.strip(), style='List Bullet')

    # Add Experience Section to document
    doc.add_paragraph("Tailored Experience", style='Heading 2')
    bullet_lines = bullet_points.split("\n")
    for line in bullet_lines:
        cleaned_line = line.strip()
        # Use NER to recognize job titles
        if not is_job_title(cleaned_line):
            # Remove leading dashes or extra spaces from experience lines
            cleaned_line = cleaned_line.lstrip('- ').strip()

            # Only add lines that are not empty and not standalone numbers or bullets
            if len(cleaned_line) > 3 and not re.match(r"^\s*[\-\•\*\d]+\s*$", cleaned_line):
                doc.add_paragraph(cleaned_line, style='List Bullet')
        else:
            doc.add_paragraph(cleaned_line)

    # # Experience to the document
    # doc.add_paragraph("Tailored Experience", style='Heading 2')
    # bullet_lines = bullet_points.split("\n")
    # for line in bullet_lines:
    #     cleaned_line = line.strip()
    #     if len(cleaned_line) > 3 and not re.match(r"^\s*[\-\•\*\d]+\s*$", line.strip()):
    #         doc.add_paragraph(line.strip(), style='List Bullet')

    # Add Education section to the document
    doc.add_paragraph("Education", style='Heading 2')
    education_lines = education.split("\n")
    for line in education_lines:
        if line.strip():
            doc.add_paragraph(line.strip(), style='List Bullet')

    # Save the document
    doc.save(file_name)
    print(f"Document saved as {file_name}")

*Note*: Function for **Education Section extraction** was adopted and modified from https://github.com/avr2002/CV-JD-Matching/blob/main/01_pdf-data-extraction.ipynb

In [35]:
# Function to extract Education from resume
def extract_education(resume_txt: str) -> str:
    education_entries = []

    lines = resume_txt.split('\n')
    for line in lines:
        if is_education_background(line):
            education_entries.append(line)

    if education_entries:
        education = '\n'.join(education_entries)
    else:
        education = "No education background found"

    return education

# Function to check if a line is education background using NER
def is_education_background(line: str) -> bool:
    doc = nlp(line)

    # Defining keywords for education
    degree_keywords = ["Bachelor", "Master", "Phd", "Doctorate", "M.Sc", "B.Sc", "MBA", "Diploma", "Associate"]
    has_degree_keyword = any(keyword in line for keyword in degree_keywords)
    for ent in doc.ents:
        if ent.label_ == "ORG" and has_degree_keyword or has_degree_keyword:
            return True
    return False

In [24]:
import gc
gc.collect()


2026

In [37]:
def main():
    api_key = userdata.get('OPENAI_API_KEY')
    model_path = "nakamoto-yama/t5-resume-generation"
    # model_path = "google/gemma-2b-it"
    print(f"Loading model and tokenizer from {model_path}")

    tokenizer, model = load_model_and_tokenizer(model_path)
    # tokenizer, model = load_gemma_model_and_tokenizer(model_path)

    # Process the experience in resume and summarize it
    experience_summary = process_resume(pdf_path, "openai", api_key)
    # print("\nResume Summary:\n", resume_summary)

    resume_summary = process_resume(pdf_path, "huggingface", api_key)


    # Test the model with a prompt to generate job description
    prompt = input("Enter a job description or title: ")
    response = generate_text(f"extract key phrases for a candidate matching the following job: {prompt}", tokenizer, model)

    response = response.replace("LB>", "{").replace("RB>", "}")
    print(f"Generated Response: {response}")

    # Extract key words from the generated response
    # key_words = extract_key_words(response)
    # print(f"Extracted Key Words: {key_words}")

    # Extract key phrases from the job description
    key_phrases = extract_key_phrases(prompt)
    print(f"Extracted Key Phrases: {key_phrases}")

    # Match experience in resume text based on key words
    # matched_experience = match_experience_in_resume(key_words, resume_summary)
    matched_experience = match_experience_to_job(key_phrases, experience_summary)

    # Format the matched experience as bullet points
    bullet_points = format_bullet_points(matched_experience)

    education = extract_education(resume_summary)
    # print(education)

    print("\nGenerated Experience Bullet Points:\n")
    print(bullet_points)

    # Generate a brief text based summary based on the bullet points
    prompt2 = "Generate a brief profile based on the following resume experience: " + bullet_points
    openai_generated_text = openai_generate_text(prompt2, api_key)
    # generate_text_with_gemma = gemma_generate_text(prompt2)

    if openai_generated_text:
        # print("\nSummary:\n")
        print(openai_generated_text)


    # Create a Word document with the bullet points and additional achievements
    create_resume_doc(bullet_points, education, openai_generated_text)

if __name__ == "__main__":
    main()